In [19]:
import pandas as pd
import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from tensorboardX import SummaryWriter

from numpy.random import choice
import pickle
import os
import random

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [2]:
seq_df = pd.read_table('05/data/family_classification_sequences.tab')
seq_df.head()

,Sequences
0,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...
1,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...
2,MQNPLPEVMSPEHDKRTTTPMSKEANKFIRELDKKPGDLAVVSDFV...
3,MDSLNEVCYEQIKGTFYKGLFGDFPLIVDKKTGCFNATKLCVLGGK...
4,MEAKNITIDNTTYNFFKFYNINQPLTNLKYLNSERLCFSNAVMGKI...


In [3]:
seq2num = {}
num2seq = {}

def build_mapping(pos, seq, num):
    if pos == 3:
        seq2num[seq] = num
        num2seq[num] = seq
        return num + 1
    for letter in range(ord('A'), ord('Z') + 1):
        num = build_mapping(pos + 1, seq + chr(letter), num)
    return num

cnt = build_mapping(0, '', 0)
assert(len(num2seq) == cnt)

In [4]:
def make_codones(sseq):
    crop = len(sseq) % 3
    cropped_seq = sseq[:-crop] if crop > 0 else sseq

    return [seq2num[cropped_seq[i:i+3]] for i in range(0, len(cropped_seq), 3)]

def seq_to3(seq):
    splittings = [make_codones(seq[i:]) for i in range(3)]
    return splittings

def create_all_codones(df):
    codones = []
    for i in range(min(df.shape[0], 1000)):
        row = df.iloc[i, :][0]
        codones.extend(seq_to3(row))
    return codones

In [5]:
def read_or_create(read_path, producer):
    if os.path.isfile(read_path):
        print('reading', read_path)
        with open(read_path, 'rb') as fp:
            return pickle.load(fp)
    result = producer()
    print('saving', read_path)
    with open(read_path, 'wb') as fp:
        pickle.dump(result, fp)
    return result

In [6]:
all_codones = read_or_create(read_path='05/data/all_codones.pickle',
                             producer= lambda: create_all_codones(seq_df))

saving 05/data/all_codones.pickle


In [7]:
presented_codones = set([item for sublist in all_codones for item in sublist])
num2cod = {}
cod2num = {}
new_index = {}
for index, cod in enumerate(presented_codones):
    num2cod[index] = num2seq[cod]
    cod2num[num2seq[cod]] = index
    new_index[cod] = index

all_codones = [[new_index[item] for item in sublist] for sublist in all_codones]
counts = [0] * len(presented_codones)

for sublist in all_codones:
    for item in sublist:
        counts[item] += 1

In [48]:
class DataProducer:
    def __init__(self, data, vocab_size, word_counts, batch_size, skip_window, num_neg_samples):
        self.data = data
        self.vocab_size = vocab_size
        self.num_neg_samples = num_neg_samples
        self.batch_size = batch_size
        self.skip_window = skip_window
        self.p_distr = self.get_distribution(word_counts)

    @staticmethod
    def get_distribution(word_counts):
        word_counts = torch.FloatTensor(word_counts)
        distr = word_counts / word_counts.sum()
        distr = distr.pow(3 / 4)
        distr = distr / distr.sum()
        return distr.numpy()

    def neg_sample(self, batch_size):
        return torch.LongTensor(
            choice(self.vocab_size, size=(batch_size, self.num_neg_samples), p=self.p_distr)
        )

    def generate_sample(self, index_words_list, context_window_size):
        """ Form training pairs according to the skip-gram model. """
        for index_words in index_words_list:
            for index, center in enumerate(index_words):
                context = random.randint(1, context_window_size)
                # get a random target before the center word
                for target in index_words[max(0, index - context): index]:
                    yield center, target
                # get a random target after the center word
                for target in index_words[index + 1: index + context + 1]:
                    yield center, target

    def get_batch(self, iterator, batch_size):
        """ Group a numerical stream into batches and yield them as Numpy arrays. """
        while True:
            center_batch = np.zeros(batch_size, dtype=np.int32)
            target_batch = np.zeros(batch_size, dtype=np.int32)
            for index in range(batch_size):
                center_batch[index], target_batch[index] = next(iterator)
            yield torch.LongTensor(center_batch), torch.LongTensor(target_batch)

    def batch_iterator(self):
        single_gen = self.generate_sample(self.data, context_window_size=self.skip_window)
        batch_gen = self.get_batch(single_gen, batch_size=self.batch_size)
        return batch_gen

In [72]:
class SkipGramModel(nn.Module):
    """ word2vec model """

    def __init__(self, vocab_size, embed_size):
        super(SkipGramModel, self).__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.embed_centers = nn.Embedding(vocab_size, embed_size)
        self.embed_contexts = nn.Embedding(vocab_size, embed_size)

    def forward(self, centers, pos_contexts, neg_contexts):
        batch_size = centers.size()[0]

        centers = self.embed_centers(centers)
        pos_contexts = self.embed_contexts(pos_contexts)
        neg_contexts = self.embed_contexts(neg_contexts)

        pos_mm = torch.bmm(centers.unsqueeze(1), pos_contexts.unsqueeze(2)).squeeze()
        pos_loss = F.logsigmoid(pos_mm).sum()

        neg_mm = torch.bmm(neg_contexts, centers.unsqueeze(2)).squeeze().sum(dim=1)
        neg_loss = F.logsigmoid(-neg_mm).sum()

        return -(pos_loss + neg_loss) / batch_size
    
    def get_embeddings(self):
        return self.embed_centers.weight

In [81]:
BATCH_SIZE = 128
SKIP_WINDOW = 12  # the context window
VOCAB_SIZE = len(presented_codones)
EMBED_SIZE = 100  # dimension of the word embedding vectors
NUM_SAMPLED = 5  # Number of negative examples to sample.
LEARNING_RATE = .05
EPOCHS = 1
SKIP_STEP = 200

In [82]:
def make_dir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass

In [83]:
def train_model(net, data_producer, optimizer, logger, epochs):
    for epoch in range(epochs):
        for i, (centers, pos_contexts) in enumerate(data_producer.batch_iterator()):
            batch_size = centers.size()[0]

            optimizer.zero_grad()
            
            loss = net(centers, pos_contexts, data_producer.neg_sample(batch_size))
            loss.backward()
            
            optimizer.step()
            
            logger.add_scalar('Loss', loss, i + 1)
            if (i + 1) % SKIP_STEP == 0:
                print("loss = {} at step {}".format(loss, i + 1))
    
    embeddings = net.get_embeddings()
    logger.add_embedding(embeddings)
    return embeddings

In [84]:
model = SkipGramModel(VOCAB_SIZE, EMBED_SIZE)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
dp = DataProducer(all_codones, VOCAB_SIZE, counts, BATCH_SIZE, SKIP_WINDOW, NUM_SAMPLED)
logger = SummaryWriter()
final_embed_matrix = train_model(model, dp, optimizer, logger, EPOCHS)

loss = 12.960344314575195 at step 200


loss = 11.518355369567871 at step 400


loss = 11.281578063964844 at step 600


loss = 12.703131675720215 at step 800


loss = 11.67991828918457 at step 1000


...

loss = 11.853784561157227 at step 38000


loss = 12.568191528320312 at step 38200


loss = 12.40804386138916 at step 38400


loss = 14.070009231567383 at step 38600


/home/egor/anaconda3/envs/07-dl/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: generator 'DataProducer.get_batch' raised StopIteration
  This is separate from the ipykernel package so we can avoid doing imports until


In [88]:
tsne = TSNE(n_components=2, random_state=42)
XX = tsne.fit_transform(final_embed_matrix.detach().numpy())

In [90]:
logger.close()

In [91]:
tsne_df = pd.DataFrame(XX, columns=['x0', 'x1'])
unique_codones = sorted(cod2num, key=cod2num.get)
tsne_df['codone'] = list(unique_codones)
tsne_df.head()

,x0,x1,codone
0,-0.204025,-14.414805,AAA
1,5.587854,6.849599,AAC
2,-2.499493,3.280342,AAD
3,-9.888508,-0.620332,AAE
4,-7.565327,-18.306129,AAF


In [92]:
def plot_tsne_df(df):
    plt.figure(figsize=(15, 10))
    plt.title('unlabeled encoding', fontsize=20)
    plt.scatter(df.x0, df.x1, s=10)
    plt.show()

In [93]:
plot_tsne_df(tsne_df)

In [96]:
filename = '05/data/acid_properties.csv'
props = pd.read_csv(filename)

In [97]:
def acid_dict(some_c, props):
    prop_by_letter = [props[props.acid == let].iloc[:, 1:] for let in some_c]   
    df_concat = pd.concat(prop_by_letter)
    res = df_concat.mean()
    dres = dict(res)
    dres['acid'] = some_c
    return dres

In [102]:
save_path = '05/data/all_acid_dicts.pickle'
producer = lambda: [acid_dict(some_c, props) for some_c in tsne_df.codone]
all_acid_dicts = read_or_create(save_path, producer)

saving 05/data/all_acid_dicts.pickle


In [105]:
all_acid_df = pd.DataFrame(all_acid_dicts)
all_acid_df.head()

,acid,hydrophobicity,mass,number_of_atoms,volume
0,AAA,1.800000,71.077900,13.000000,88.600000
1,AAC,2.033333,81.766233,13.333333,95.233333
2,AAD,0.033333,85.747733,14.000000,96.100000
3,AAE,0.033333,90.423267,15.000000,105.200000
4,AAF,2.133333,96.443233,16.333333,122.366667


In [106]:
final_df = all_acid_df.join(tsne_df.set_index('codone'), on='acid')
final_df.head()

,acid,hydrophobicity,mass,number_of_atoms,volume,x0,x1
0,AAA,1.800000,71.077900,13.000000,88.600000,-0.204025,-14.414805
1,AAC,2.033333,81.766233,13.333333,95.233333,5.587854,6.849599
2,AAD,0.033333,85.747733,14.000000,96.100000,-2.499493,3.280342
3,AAE,0.033333,90.423267,15.000000,105.200000,-9.888508,-0.620332
4,AAF,2.133333,96.443233,16.333333,122.366667,-7.565327,-18.306129


In [107]:
def plot_embedding_properties(final_df):
    plt.figure(figsize=(25, 20))
    for i, p in enumerate(['hydrophobicity', 'mass', 'number_of_atoms', 'volume']):
        plt.subplot(2,2,i+1)
        plt.title(p, fontsize=25)
        plt.scatter(final_df.x0, final_df.x1, c=final_df[p], s=10)
    plt.show()

In [108]:
plot_embedding_properties(final_df)

In [109]:
filename = '05/data/nice_embed_tsne.csv'
gensim_tsne_df = pd.read_csv(filename, index_col=0)
gensim_tsne_df.columns = ['x0', 'x1', 'codone']

In [110]:
plot_tsne_df(gensim_tsne_df)

In [111]:
final_df_nice = all_acid_df.join(gensim_tsne_df.set_index('codone'), on='acid')

In [112]:
plot_embedding_properties(final_df_nice)

In [12]:
cnt_batches = 0
cnt_examples = 0
for batch in data_producer.batch_iterator():
    cnt_batches += 1
    cnt_examples += len(batch[0])
print(cnt_batches)
print(cnt_examples)

38706
4954368


/home/egor/anaconda3/envs/07-dl/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: generator 'DataProducer.get_batch' raised StopIteration
  This is separate from the ipykernel package so we can avoid doing imports until
